##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 체크포인트 저장 항목 마이그레이션하기

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/migrate/checkpoint_saver"><img src="https://www.tensorflow.org/images/tf_logo_32px.png"> TensorFlow.org에서 보기</a></td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/guide/migrate/checkpoint_saver.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행하기</a>
</td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/guide/migrate/checkpoint_saver.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a></td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/guide/migrate/checkpoint_saver.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드하기</a>
</td>
</table>


"최고" 모델 또는 모델 가중치/매개변수를 지속적으로 저장하면 많은 이점이 있습니다. 이러한 이점에는 교육 진행 상황을 추적하고 다른 저장된 상태에서 저장된 모델을 로드하기 등이 있습니다.

TensorFlow 1에서 `tf.estimator.Estimator` API로 훈련/검증을 진행하는 동안에 체크포인트 저장을 구성하려면 `tf.estimator.RunConfig`에서 일정을 지정하거나 `tf.estimator.CheckpointSaverHook`를 사용합니다. 이 가이드는 이 워크플로에서 TensorFlow 2 Keras API로 마이그레이션하는 방법을 보여줍니다.

TensorFlow 2에서는 다음과 같은 다양한 방법으로 `tf.keras.callbacks.ModelCheckpoint`를 구성할 수 있습니다.

- `save_best_only=True` 매개변수를 사용하여 모니터링되는 메트릭에 따라 '최고' 버전을 저장합니다. 예를 들어 여기서 `monitor`는 `'loss'`, `'val_loss'`, `'accuracy', or `'val_accuracy'`일 수 있습니다.
- 정 빈도로 계속 저장합니다(`save_freq` 인수 사용).
- `save_weights_only`를 `True`로 설정하여 전체 모델 대신 가중치/매개변수만 저장합니다.

자세한 내용은 `tf.keras.callbacks.ModelCheckpoint` API 문서 및 [모델 저장 및 로드하기](../../tutorials/keras/save_and_load.ipynb) 튜토리얼의 *훈련하는 동안 체크포인트 저장하기* 섹션을 참고합니다. [Keras 모델 저장 및 로드하기](https://www.tensorflow.org/guide/keras/save_and_serialize) 가이드의 *TF 체크포인트 형식* 섹션에서 체크포인트 형식에 대한 자세한 내용을 확인할 수 있습니다. 또한 내결함성을 추가하기 위해 수동 체크포인트에 `tf.keras.callbacks.BackupAndRestore` 또는 `tf.train.Checkpoint`를 사용할 수 있습니다. [내결함성 마이그레이션 가이드](fault_tolerance.ipynb)에서 자세히 알아보세요.

Keras [콜백](https://www.tensorflow.org/guide/keras/custom_callback)은 내장 Keras `Model.fit`/`Model.evaluate`/`Model.predict` API에서 훈련/평가/예측을 진행하는 동안 서로 다른 포인트에서 호출되는 객체입니다. 가이드 끝에 있는 *다음 단계* 섹션에서 자세히 알아보세요.

## 설치하기

데모를 위해 가져오기 및 간단한 데이터세트로 시작해 보겠습니다.

In [ ]:
import tensorflow.compat.v1 as tf1
import tensorflow as tf
import numpy as np
import tempfile

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

## TensorFlow 1: tf.estimator API를 사용하여 체크포인트 저장하기

이 TensorFlow 1 예제는 `tf.estimator.Estimator` API를 사용하여 훈련/평가를 진행하는 동안 모든 단계에서 체크포인트를 저장하도록 `tf.estimator.RunConfig`를 구성하는 방법을 보여줍니다.

In [ ]:
feature_columns = [tf1.feature_column.numeric_column("x", shape=[28, 28])]

config = tf1.estimator.RunConfig(save_summary_steps=1,
                                 save_checkpoints_steps=1)

path = tempfile.mkdtemp()

classifier = tf1.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[256, 32],
    optimizer=tf1.train.AdamOptimizer(0.001),
    n_classes=10,
    dropout=0.2,
    model_dir=path,
    config = config
)

train_input_fn = tf1.estimator.inputs.numpy_input_fn(
    x={"x": x_train},
    y=y_train.astype(np.int32),
    num_epochs=10,
    batch_size=50,
    shuffle=True,
)

test_input_fn = tf1.estimator.inputs.numpy_input_fn(
    x={"x": x_test},
    y=y_test.astype(np.int32),
    num_epochs=10,
    shuffle=False
)

train_spec = tf1.estimator.TrainSpec(input_fn=train_input_fn, max_steps=10)
eval_spec = tf1.estimator.EvalSpec(input_fn=test_input_fn,
                                   steps=10,
                                   throttle_secs=0)

tf1.estimator.train_and_evaluate(estimator=classifier,
                                train_spec=train_spec,
                                eval_spec=eval_spec)

In [ ]:
%ls {classifier.model_dir}

## TensorFlow 2: Model.fit을 위해 Keras 콜백을 사용하여 체크포인트 저장하기

TensorFlow 2에서 훈련/평가에 대해 내장 Keras `Model.fit`(또는 `Model.evaluate`)을 사용하는 경우 `tf.keras.callbacks.ModelCheckpoint`를 구성하고 `Model.fit`(또는 `Model.evaluate`)의 `callbacks` 매개변수로 전달할 수 있습니다(API 문서 및 [내장 메서드를 사용하여 훈련 및 평가하기](https://www.tensorflow.org/guide/keras/train_and_evaluate) 가이드의 *콜백 사용하기* 섹션에서 자세한 내용 확인).

아래 예제에서는 `tf.keras.callbacks.ModelCheckpoint` 콜백을 사용하여 임시 디렉터리에 체크포인트를 저장합니다.

In [ ]:
def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],
              steps_per_execution=10)

log_dir = tempfile.mkdtemp()

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=log_dir)

model.fit(x=x_train,
          y=y_train,
          epochs=10,
          validation_data=(x_test, y_test),
          callbacks=[model_checkpoint_callback])

In [ ]:
%ls {model_checkpoint_callback.filepath}

## 다음 단계

체크포인트에 대한 자세한 내용:

- API 문서: `tf.keras.callbacks.ModelCheckpoint`
- 튜토리얼: [모델 저장 및 로드하기](../../tutorials/keras/save_and_load.ipynb)(*훈련하는 동안 체크포인트 저장하기* 섹션)
- 가이드: [Keras 모델 저장 및 로드하기](https://www.tensorflow.org/guide/keras/save_and_serialize)(*TF 체크포인트 형식* 섹션)

콜백에 대한 자세한 내용:

- API 문서: `tf.keras.callbacks.Callback`
- 가이드: [자신만의 콜백 작성하기](https://www.tensorflow.org/guide/keras/guide/keras/custom_callback)
- 가이드: [내장 메서드를 사용하여 훈련 및 평가하기](https://www.tensorflow.org/guide/keras/train_and_evaluate)(*콜백 사용하기* 섹션)

다음과 같은 마이그레이션 관련 리소스도 유용할 수 있습니다.

- [내결함성 마이그레이션 가이드](fault_tolerance.ipynb): `Model.fit`의 경우 `tf.keras.callbacks.BackupAndRestore`, 사용자 정의 훈련 루프의 경우 `tf.train.Checkpoint`와 `tf.train.CheckpointManager` API
- [조기 중단 마이그레이션 가이드](early_stopping.ipynb): 내장 조기 중단 콜백인 `tf.keras.callbacks.EarlyStopping`
- [TensorBoard 마이그레이션 가이드](tensorboard.ipynb): TensorBoard를 사용하여 메트릭 추적 및 표시하기
- [LoggingTensorHook 및 StopAtStepHook에서 Keras 콜백으로의 마이그레이션 가이드](logging_stop_hook.ipynb)
- [SessionRunHook에서 Keras 콜백을 수행하기 위한 가이드](sessionrunhook_callback.ipynb)